In [65]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import requests
from io import StringIO as StringIO
import re
import pandas as pd
import numpy as np
from collections import defaultdict

print (__version__)

init_notebook_mode(connected=True)

3.10.0


In [54]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [66]:
path=r"c:\Results\100users10000sec3600tt02loop10.jtl"

df = pd.read_csv(path)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100181 entries, 0 to 100180
Data columns (total 17 columns):
timeStamp          100181 non-null int64
elapsed            100181 non-null int64
label              100181 non-null object
responseCode       100106 non-null object
responseMessage    100181 non-null object
threadName         97906 non-null object
dataType           58794 non-null object
success            100181 non-null bool
failureMessage     0 non-null float64
bytes              100181 non-null int64
sentBytes          100181 non-null int64
grpThreads         100181 non-null int64
allThreads         100181 non-null int64
URL                58814 non-null object
Latency            100181 non-null int64
IdleTime           100181 non-null int64
Connect            100181 non-null int64
dtypes: bool(1), float64(1), int64(9), object(6)
memory usage: 12.3+ MB


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



,timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,failureMessage,bytes,sentBytes,grpThreads,allThreads,URL,Latency,IdleTime,Connect
0,1561593913731,832,/discovery/api,200,OK,API 1-1,text,True,NaN,5910,212,1,1,https://studio.bluebeam-dev.com/discovery/api,831,0,676
1,1561593911325,832,00_Discovery,200,"Number of samples in transaction : 1, number o...",API 1-1,NaN,True,NaN,5910,212,1,1,NaN,831,2412,676
2,1561593916874,1029,token,200,OK,API 1-1,text,True,NaN,867,349,1,1,https://revuapi-d-xx-ue1.bluebeam-dev.com/token,1029,0,287
3,1561593914570,1029,01_Get_token,200,"Number of samples in transaction : 1, number o...",API 1-1,NaN,True,NaN,867,349,1,1,NaN,1029,2346,287
4,1561593920516,2,Debug Sampler,200,OK,API 1-1,text,True,NaN,1201,0,1,1,NaN,0,0,0


In [67]:
useful_col_names=['timeStamp', 'label', 'elapsed', 'success']
df=df[useful_col_names]

transaction_pattern=r'^\d{2,3}_.+'
tr_df=df[df.label.str.contains(transaction_pattern)]

tr_df.head(20)

,timeStamp,label,elapsed,success
1,1561593911325,00_Discovery,832,True
3,1561593914570,01_Get_token,1029,True
7,1561593920518,02_Configuration_and_alerts,489,True
9,1561593922610,03_Identity,303,True
11,1561593921320,00_Discovery,354,True
16,1561593923977,01_Get_token,866,True
24,1561593929448,02_Configuration_and_alerts,389,True
27,1561593931440,03_Identity,89,True
32,1561593931317,00_Discovery,298,True
40,1561593933918,01_Get_token,821,True


In [68]:
stats_df=tr_df[tr_df['success']].groupby("label").elapsed.agg([np.size, np.mean, np.std, np.min, np.max, percentile(50), percentile(90), percentile(95), percentile(99)])
stats_df.head(10)

,size,mean,std,amin,amax,percentile_50,percentile_90,percentile_95,percentile_99
label,,,,,,,,,
00_Discovery,100,304.010000,64.145116,267,832,293.0,318.1,331.55,609.25
01_Get_token,100,2933.500000,3123.403309,662,13793,994.5,7058.8,9104.15,13585.10
02_Configuration_and_alerts,2468,3710.496353,3922.785830,155,24906,1824.5,9485.2,11463.40,15511.72
03_Identity,2464,1837.043831,2814.951103,71,21433,390.0,6034.2,7963.20,12455.05
04_Join_session,2436,11619.172414,6368.632385,903,34942,11961.5,19715.0,22034.50,25830.00
05_Send_chat_message,2430,2217.931687,2838.224901,138,19881,730.5,6289.5,8029.95,12274.18
06_Projects,2427,1903.714462,2755.743604,71,18114,426.0,6126.2,7890.70,11280.56
07_Documents_calls,2419,5928.763952,4657.546770,447,26668,5415.0,12534.2,14648.00,18503.40
08_Add_markup,23913,2285.700498,2939.752486,167,20208,786.0,6587.2,8478.20,12776.88


In [69]:
count_tr_df=tr_df.groupby('label', as_index=False).elapsed.mean()
count_tr_df.head(20)

,label,elapsed
0,00_Discovery,304.010000
1,01_Get_token,2933.500000
2,02_Configuration_and_alerts,3709.305421
3,03_Identity,1837.210697
4,04_Join_session,11627.447261
5,05_Send_chat_message,2217.088670
6,06_Projects,1903.505761
7,07_Documents_calls,5927.854545
8,08_Add_markup,2284.722412
9,09_get_markups_from_other_users,66.794526


In [70]:
grouped=tr_df.sort_values(by=['timeStamp']).groupby(["label", "success"])
traces=[]
for group_name,grouped_df in grouped:
    
    trace_tr = go.Scatter(
        x = grouped_df.timeStamp,
        y = grouped_df.elapsed,
        mode = 'lines',
        name = group_name[0]
    )
    traces.append(trace_tr)


layout = go.Layout(
    title='Transaction time',
)   
    
fig = go.Figure(data = traces, layout = layout)
iplot(fig)

In [71]:
transaction_names=tr_df['label'].unique()
print(transaction_names)

['00_Discovery' '01_Get_token' '02_Configuration_and_alerts' '03_Identity'
 '04_Join_session' '05_Send_chat_message' '06_Projects'
 '07_Documents_calls' '08_Add_markup' '09_get_markups_from_other_users']


In [72]:
N = len(transaction_names)

# cгенерируем цвета для визуализации
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

data = [{
    'y': tr_df[tr_df.label==transaction_names[i]].elapsed, 
    'type':'box',
    'marker':{'color': c[i]},
    'name': transaction_names[i]
    } for i in range(N)]

layout = go.Layout(
    title='Transaction Time',
    yaxis = {'title': 'Time,ms'}
)   

fig = go.Figure(data = data, layout = layout)
iplot(fig)
